In [5]:
import time

In [2]:
import random
import math

# Fitness function: number of diagonal attacks
def fitness(chromosome):
    attacks = 0
    n = len(chromosome)
    for i in range(n):
        for j in range(i + 1, n):
            if abs(chromosome[i] - chromosome[j]) == abs(i - j):  # diagonal conflict
                attacks += 1
    return attacks

# Swap mutation
def mutate(chromosome):
    i, j = random.sample(range(len(chromosome)), 2)
    chromosome[i], chromosome[j] = chromosome[j], chromosome[i]
    return chromosome

# Order Crossover (OX)
def crossover(p1, p2):
    size = len(p1)
    a, b = sorted(random.sample(range(size), 2))
    hole = [item for item in p2 if item not in p1[a:b]]
    return hole[:a] + p1[a:b] + hole[a:]

# Simulated Annealing for Local Refinement
def simulated_annealing(chromosome, initial_temp=100.0, cooling_rate=0.99, iterations=100):
    current = chromosome[:]
    current_fit = fitness(current)
    temp = initial_temp

    for _ in range(iterations):
        if current_fit == 0:
            break
        candidate = mutate(current[:])
        candidate_fit = fitness(candidate)
        delta = candidate_fit - current_fit

        if delta < 0 or random.random() < math.exp(-delta / temp):
            current = candidate
            current_fit = candidate_fit

        temp *= cooling_rate

    return current

# Hybrid GA + SA
def hybrid_n_queens(n, pop_size=100, generations=1000, elite_size=10, sa_top_k=5):
    # Initial population
    population = [random.sample(range(n), n) for _ in range(pop_size)]

    for gen in range(generations):
        # Sort by fitness
        population = sorted(population, key=fitness)

        # Check for solution
        if fitness(population[0]) == 0:
            return population[0]

        # Apply Simulated Annealing to top individuals
        for i in range(sa_top_k):
            population[i] = simulated_annealing(population[i])

        # Elitism
        next_gen = population[:elite_size]

        # Generate rest of the population
        while len(next_gen) < pop_size:
            parent1, parent2 = random.sample(population[:50], 2)
            child = crossover(parent1, parent2)
            if random.random() < 0.3:
                child = mutate(child)
            next_gen.append(child)

        population = next_gen

    return None


In [25]:
n = 13
start = time.time()
solution = hybrid_n_queens(n)
end = time.time()
if solution:
    print(f"Hybrid GA-SA Solution for {n}-Queens: {solution}")
    for i in range(n):
        row = ['.'] * n
        row[solution[i]] = 'Q'
        print(''.join(row))
else:
    print("No solution found.")
print(f"Total Time Taken to find a solution is: {end-start} seconds")


Hybrid GA-SA Solution for 13-Queens: [3, 6, 9, 11, 2, 4, 10, 8, 5, 1, 12, 7, 0]
...Q.........
......Q......
.........Q...
...........Q.
..Q..........
....Q........
..........Q..
........Q....
.....Q.......
.Q...........
............Q
.......Q.....
Q............
Total Time Taken to find a solution is: 0.5579831600189209 seconds


In [4]:
def is_safe(board, row, col, n):
    # Check column
    for i in range(row):
        if board[i][col] == 1:
            return False

    # Check upper left diagonal
    for i, j in zip(range(row - 1, -1, -1), range(col - 1, -1, -1)):
        if board[i][j] == 1:
            return False

    # Check upper right diagonal
    for i, j in zip(range(row - 1, -1, -1), range(col + 1, n)):
        if board[i][j] == 1:
            return False

    return True


def solve_n_queens_util(board, row, n, solutions):
    if row == n:
        solution = [''.join('Q' if cell else '.' for cell in row) for row in board]
        solutions.append(solution)
        return

    for col in range(n):
        if is_safe(board, row, col, n):
            board[row][col] = 1
            solve_n_queens_util(board, row + 1, n, solutions)
            board[row][col] = 0


def solve_n_queens_conventional(n):
    board = [[0] * n for _ in range(n)]
    solutions = []
    solve_n_queens_util(board, 0, n, solutions)
    return solutions


# Example

n = 8
conventional_solutions = solve_n_queens_conventional(n)
print(f"Number of solutions for {n}-Queens (Conventional): {len(conventional_solutions)}")


Number of solutions for 8-Queens (Conventional): 92


In [8]:
def solve_n_queens_first(board, row, n):
    if row == n:
        return [''.join('Q' if cell else '.' for cell in r) for r in board]

    for col in range(n):
        if is_safe(board, row, col, n):
            board[row][col] = 1
            result = solve_n_queens_first(board, row + 1, n)
            if result:
                return result
            board[row][col] = 0
    return None

def solve_first_n_queen(n):
    board = [[0] * n for _ in range(n)]
    return solve_n_queens_first(board, 0, n)

solution = solve_first_n_queen(8)
for row in solution:
    print(row)


Q.......
....Q...
.......Q
.....Q..
..Q.....
......Q.
.Q......
...Q....


In [9]:
import time

def is_safe(board, row, col, n):
    # Check column
    for i in range(row):
        if board[i][col] == 1:
            return False

    # Check upper left diagonal
    for i, j in zip(range(row - 1, -1, -1), range(col - 1, -1, -1)):
        if board[i][j] == 1:
            return False

    # Check upper right diagonal
    for i, j in zip(range(row - 1, -1, -1), range(col + 1, n)):
        if board[i][j] == 1:
            return False

    return True


def solve_n_queens_first(board, row, n):
    if row == n:
        # Found one valid solution — return it
        return [''.join('Q' if cell else '.' for cell in r) for r in board]

    for col in range(n):
        if is_safe(board, row, col, n):
            board[row][col] = 1
            result = solve_n_queens_first(board, row + 1, n)
            if result:
                return result  # Return early on first found solution
            board[row][col] = 0

    return None


def solve_first_n_queen(n):
    board = [[0] * n for _ in range(n)]
    return solve_n_queens_first(board, 0, n)


# --- Run and time it ---
n = 8

start_time = time.time()

optimal_solution = solve_first_n_queen(n)

end_time = time.time()
elapsed_time = end_time - start_time

# --- Output ---
print(f"\nFirst Optimal Solution for {n}-Queens (Conventional Backtracking):\n")
for row in optimal_solution:
    print(row)

print(f"\n🕒 Time Taken: {elapsed_time:.6f} seconds")



✅ First Optimal Solution for 8-Queens (Conventional Backtracking):

Q.......
....Q...
.......Q
.....Q..
..Q.....
......Q.
.Q......
...Q....

🕒 Time Taken: 0.001896 seconds


In [10]:
import time

def is_safe(board, row, col, n):
    # Check column
    for i in range(row):
        if board[i][col] == 1:
            return False

    # Check upper left diagonal
    for i, j in zip(range(row - 1, -1, -1), range(col - 1, -1, -1)):
        if board[i][j] == 1:
            return False

    # Check upper right diagonal
    for i, j in zip(range(row - 1, -1, -1), range(col + 1, n)):
        if board[i][j] == 1:
            return False

    return True

def solve_n_queens_util(board, row, n, solutions):
    if row == n:
        solution = [''.join('Q' if cell else '.' for cell in r) for r in board]
        solutions.append(solution)
        return

    for col in range(n):
        if is_safe(board, row, col, n):
            board[row][col] = 1
            solve_n_queens_util(board, row + 1, n, solutions)
            board[row][col] = 0

def solve_n_queens_conventional(n):
    board = [[0] * n for _ in range(n)]
    solutions = []
    solve_n_queens_util(board, 0, n, solutions)
    return solutions

# ---- Run and time the full solution ----
n = 8  # You can increase this to 10, 12, etc.

start_time = time.time()

conventional_solutions = solve_n_queens_conventional(n)

end_time = time.time()
elapsed_time = end_time - start_time

# ---- Output ----
print(f"\n✅ Number of Optimal Solutions for {n}-Queens (Conventional): {len(conventional_solutions)}")
print(f"🕒 Time Taken: {elapsed_time:.6f} seconds\n")

# Optional: Print first few solutions
for i, solution in enumerate(conventional_solutions[:3], 1):
    print(f"Solution #{i}:")
    for row in solution:
        print(row)
    print()



✅ Number of Optimal Solutions for 8-Queens (Conventional): 92
🕒 Time Taken: 0.021511 seconds

Solution #1:
Q.......
....Q...
.......Q
.....Q..
..Q.....
......Q.
.Q......
...Q....

Solution #2:
Q.......
.....Q..
.......Q
..Q.....
......Q.
...Q....
.Q......
....Q...

Solution #3:
Q.......
......Q.
...Q....
.....Q..
.......Q
.Q......
....Q...
..Q.....



In [26]:
import time

def is_safe(board, row, col, n):
    # Check column
    for i in range(row):
        if board[i][col] == 1:
            return False

    # Check upper left diagonal
    for i, j in zip(range(row - 1, -1, -1), range(col - 1, -1, -1)):
        if board[i][j] == 1:
            return False

    # Check upper right diagonal
    for i, j in zip(range(row - 1, -1, -1), range(col + 1, n)):
        if board[i][j] == 1:
            return False

    return True

def solve_first_solution(board, row, n):
    if row == n:
        # Return the board as a solution
        return [''.join('Q' if cell else '.' for cell in r) for r in board]

    for col in range(n):
        if is_safe(board, row, col, n):
            board[row][col] = 1
            result = solve_first_solution(board, row + 1, n)
            if result:  # If a solution is found, return it immediately
                return result
            board[row][col] = 0
    return None

def solve_n_queens_first(n):
    board = [[0]*n for _ in range(n)]
    return solve_first_solution(board, 0, n)

# ---- Execution & Timing ----
n = 13  # Change N as needed
start_time = time.time()
first_solution = solve_n_queens_first(n)
end_time = time.time()

# ---- Output ----
if first_solution:
    print(f"✅ First Optimal Solution for {n}-Queens:\n")
    for row in first_solution:
        print(row)
else:
    print("No solution found.")

print(f"\n🕒 Time Taken: {end_time - start_time:.6f} seconds")


✅ First Optimal Solution for 13-Queens:

Q............
..Q..........
....Q........
.Q...........
........Q....
...........Q.
.........Q...
............Q
...Q.........
.....Q.......
.......Q.....
..........Q..
......Q......

🕒 Time Taken: 0.001089 seconds


In [27]:
import time

def time_conventional(n):
    start = time.time()
    solve_n_queens_conventional(n)
    return time.time() - start

def time_hybrid(n, runs=5):
    total_time = 0
    valid_count = 0
    for _ in range(runs):
        start = time.time()
        solution = solve_n_queens_ga_sa(n)
        duration = time.time() - start
        total_time += duration
        if is_valid_solution(solution):  # You can define this to check 0 conflicts
            valid_count += 1
    return total_time / runs, valid_count


In [32]:
import time
import random

# ----- Dummy GA+SA Method Placeholder -----
# Replace this with your actual hybrid GA+SA implementation
def solve_n_queens_ga_sa(n):
    # Fake logic for demonstration – replace this with your real code
    board = [-1] * n
    for i in range(n):
        board[i] = random.randint(0, n-1)
    return board  # Returns list: index = row, value = column

# ----- Conventional Solver -----
def is_safe(board, row, col, n):
    for i in range(row):
        if board[i][col] == 1:
            return False
    for i, j in zip(range(row-1, -1, -1), range(col-1, -1, -1)):
        if board[i][j] == 1:
            return False
    for i, j in zip(range(row-1, -1, -1), range(col+1, n)):
        if board[i][j] == 1:
            return False
    return True

def solve_util(board, row, n, solutions, limit=1):
    if row == n:
        solution = [''.join('Q' if cell else '.' for cell in r) for r in board]
        solutions.append(solution)
        return len(solutions) < limit  # Continue if we haven't reached limit
    for col in range(n):
        if is_safe(board, row, col, n):
            board[row][col] = 1
            if not solve_util(board, row + 1, n, solutions, limit):
                return False
            board[row][col] = 0
    return True

def solve_n_queens_conventional(n, limit=1):
    board = [[0]*n for _ in range(n)]
    solutions = []
    solve_util(board, 0, n, solutions, limit)
    return solutions[0] if solutions else None

# ----- Check Validity -----
def is_valid_solution(solution):
    n = len(solution)
    seen_cols = set()
    seen_diag1 = set()
    seen_diag2 = set()
    for row, col in enumerate(solution):
        if col in seen_cols or (row - col) in seen_diag1 or (row + col) in seen_diag2:
            return False
        seen_cols.add(col)
        seen_diag1.add(row - col)
        seen_diag2.add(row + col)
    return True

# ----- Comparison Script -----
def compare_methods(n, ga_sa_runs=5):
    print(f"🧪 Comparing N = {n}\n")

    # --- Conventional ---
    start = time.time()
    conventional_solution = solve_n_queens_conventional(n)
    end = time.time()
    conventional_time = end - start
    print("✅ Conventional method:")
    print(f"   Time Taken: {conventional_time:.6f} seconds")
    print(f"   Valid: {'Yes' if conventional_solution else 'No'}")

    # --- GA + SA ---
    ga_valid_count = 0
    ga_total_time = 0
    print("\n🤖 Hybrid GA + SA method ({} runs):".format(ga_sa_runs))
    for i in range(ga_sa_runs):
        start = time.time()
        ga_solution = solve_n_queens_ga_sa(n)
        end = time.time()
        duration = end - start
        ga_total_time += duration
        if is_valid_solution(ga_solution):
            ga_valid_count += 1
        print(f"   Run {i+1}: Time = {duration:.6f}s | Valid: {'Yes' if is_valid_solution(ga_solution) else 'No'}")

    ga_avg_time = ga_total_time / ga_sa_runs
    print(f"\n📊 GA+SA Average Time: {ga_avg_time:.6f} seconds")
    print(f"📈 Valid Solutions Found: {ga_valid_count}/{ga_sa_runs}")

# ----- Run the Comparison -----
compare_methods(n=8, ga_sa_runs=5)


🧪 Comparing N = 8

✅ Conventional method:
   Time Taken: 0.000572 seconds
   Valid: Yes

🤖 Hybrid GA + SA method (5 runs):
   Run 1: Time = 0.000009s | Valid: No
   Run 2: Time = 0.000005s | Valid: No
   Run 3: Time = 0.000005s | Valid: No
   Run 4: Time = 0.000005s | Valid: No
   Run 5: Time = 0.000004s | Valid: No

📊 GA+SA Average Time: 0.000006 seconds
📈 Valid Solutions Found: 0/5


In [36]:
import time
import random
import math

# ----- Hybrid Genetic Algorithm + Simulated Annealing -----

def fitness(solution):
    """Count the number of conflicting pairs of queens (diagonal only)."""
    n = len(solution)
    conflicts = 0
    for i in range(n):
        for j in range(i + 1, n):
            if abs(solution[i] - solution[j]) == abs(i - j):
                conflicts += 1
    return -conflicts  # More negative = worse

def mutate(solution):
    """Randomly swap two elements (columns of two rows)."""
    a, b = random.sample(range(len(solution)), 2)
    solution[a], solution[b] = solution[b], solution[a]
    return solution

def simulated_annealing(solution, temp=1.0, cooling_rate=0.95, max_iter=100):
    current = solution[:]
    current_score = fitness(current)
    for _ in range(max_iter):
        candidate = mutate(current[:])
        candidate_score = fitness(candidate)
        delta = candidate_score - current_score
        if delta > 0 or random.random() < math.exp(delta / temp):
            current, current_score = candidate, candidate_score
        temp *= cooling_rate
    return current

def solve_n_queens_ga_sa(n, population_size=100, generations=200):
    population = [random.sample(range(n), n) for _ in range(population_size)]

    for _ in range(generations):
        population.sort(key=lambda sol: fitness(sol), reverse=True)
        if fitness(population[0]) == 0:
            return population[0]
        survivors = population[:population_size // 2]
        children = []
        while len(children) < population_size // 2:
            parent = random.choice(survivors)[:]
            child = mutate(parent)
            child = simulated_annealing(child)
            children.append(child)
        population = survivors + children

    return max(population, key=fitness)


# ----- Conventional Backtracking Solver -----

def is_safe(board, row, col, n):
    for i in range(row):
        if board[i][col] == 1:
            return False
    for i, j in zip(range(row-1, -1, -1), range(col-1, -1, -1)):
        if board[i][j] == 1:
            return False
    for i, j in zip(range(row-1, -1, -1), range(col+1, n)):
        if board[i][j] == 1:
            return False
    return True

def solve_util(board, row, n, solutions, limit=1):
    if row == n:
        solution = [''.join('Q' if cell else '.' for cell in r) for r in board]
        solutions.append(solution)
        return len(solutions) < limit
    for col in range(n):
        if is_safe(board, row, col, n):
            board[row][col] = 1
            if not solve_util(board, row + 1, n, solutions, limit):
                return False
            board[row][col] = 0
    return True

def solve_n_queens_conventional(n, limit=1):
    board = [[0]*n for _ in range(n)]
    solutions = []
    solve_util(board, 0, n, solutions, limit)
    return solutions[0] if solutions else None


# ----- Utility to Check Validity of GA+SA Solution -----

def is_valid_solution(solution):
    n = len(solution)
    seen_cols = set()
    seen_diag1 = set()
    seen_diag2 = set()
    for row, col in enumerate(solution):
        if col in seen_cols or (row - col) in seen_diag1 or (row + col) in seen_diag2:
            return False
        seen_cols.add(col)
        seen_diag1.add(row - col)
        seen_diag2.add(row + col)
    return True


# ----- Comparison Runner -----

def compare_methods(n, ga_sa_runs=5):
    print(f"🧪 Comparing N = {n}\n")

    # --- Conventional ---
    start = time.time()
    conventional_solution = solve_n_queens_conventional(n)
    end = time.time()
    conventional_time = end - start
    print("✅ Conventional method:")
    print(f"   Time Taken: {conventional_time:.6f} seconds")
    print(f"   Valid: {'Yes' if conventional_solution else 'No'}")

    # --- GA + SA ---
    ga_valid_count = 0
    ga_total_time = 0
    print("\n🤖 Hybrid GA + SA method ({} runs):".format(ga_sa_runs))
    for i in range(ga_sa_runs):
        start = time.time()
        ga_solution = solve_n_queens_ga_sa(n)
        end = time.time()
        duration = end - start
        ga_total_time += duration
        valid = is_valid_solution(ga_solution)
        if valid:
            ga_valid_count += 1
        print(f"   Run {i+1}: Time = {duration:.6f}s | Valid: {'Yes' if valid else 'No'}")

    ga_avg_time = ga_total_time / ga_sa_runs
    print(f"\n📊 GA+SA Average Time: {ga_avg_time:.6f} seconds")
    print(f"📈 Valid Solutions Found: {ga_valid_count}/{ga_sa_runs}")


# ----- Run the Comparison -----

compare_methods(n=25, ga_sa_runs=5)


🧪 Comparing N = 25

✅ Conventional method:
   Time Taken: 1.230918 seconds
   Valid: Yes

🤖 Hybrid GA + SA method (5 runs):
   Run 1: Time = 0.201164s | Valid: Yes
   Run 2: Time = 0.587729s | Valid: Yes
   Run 3: Time = 0.979306s | Valid: Yes
   Run 4: Time = 1.609694s | Valid: Yes
   Run 5: Time = 0.979379s | Valid: Yes

📊 GA+SA Average Time: 0.871455 seconds
📈 Valid Solutions Found: 5/5
